In [ ]:
import numpy as np
from GP_regression import * 
import scipy.stats as sc


In [ ]:
### variables = [flux_e, flux_i, RA_frac, coupling, lpi_loss_frac] ###

sample_space = [sc.uniform(0.00,0.3), sc.uniform(0.35,0.3), sc.uniform(0.15,0.3),
                sc.uniform(0.65,0.3),sc.uniform(0.05,0.3)]

bounds = np.array([[0.0, 0.1], [0.4, 0.6], [0.2, 0.4], [0.75, 0.95], [0.05, 0.35]])


In [ ]:
LH = LHC(sample_space=sample_space, bounds=bounds,\
         calib_samp=500, GP_samp=100000)


In [ ]:
LH.calibrate_samples(new_calib = False, best_data = True)

In [ ]:
# LH.plot_prior_LHC()

In [ ]:
LH_samples = LH.X_data
log_L_results_P1 = LH.Y_P1_data
log_L_results_P2 = LH.Y_P2_data
log_L_results_P3 = LH.Y_P3_data
ST_results_P2 = LH.Y_ST_P2_data
ST_results_P3 = LH.Y_ST_P3_data




In [ ]:
GPs_log_L = {}

profiles = [1, 2, 3]
outputs = [log_L_results_P1, log_L_results_P2, log_L_results_P3]

for profile, output in zip(profiles, outputs):
    
    GPs_log_L[profile]  = GP_reg(kernel='Exponential', samples=LH_samples, \
                           output=output, train_frac=0.99, restarts = 50)

In [ ]:
fnames = ['GP_calib/log_L_P1_GP.pickle', 'GP_calib/log_L_P2_GP.pickle', 'GP_calib/log_L_P3_GP.pickle']
titles = [r'Profile 1', r'Profile 2', r'Profile 3']

for profile, fname, title in zip(profiles, fnames, titles):

    # if (profile == 1 or profile == 3):
    #     continue
    
    GPs_log_L[profile].test_train()
    GPs_log_L[profile].regression()
    GPs_log_L[profile].m.pickle(fname)
    print(GPs_log_L[profile].len_scales)
    GPs_log_L[profile].test_train_plot(title = title, likelihood = True)



In [ ]:
predict_samples = LH.GP_samp_space

params = {}

for profile in profiles:
    # if (profile == 1 or profile == 3):
    #     continue
    

    log_L_p, V_p = GPs_log_L[profile].GP_predict(X = predict_samples)

    index = int(np.where(log_L_p == np.max(log_L_p))[0])
    print(index)

    params[profile] = predict_samples[index, :]


print(params)

In [ ]:
# profile 1
# theta = [0.06, 0.42, 0.253, 0.71, 0.34]
shk_vel(params=params[1], profile=1, plot=True)

# shk_vel(params=theta, profile=1, plot=True)

In [ ]:
# profile 2
#theta = [0.062821 , 0.477645 , 0.244835 , 0.7885375, 0.2106275]
shk_vel(params=params[2], profile=2, plot=True)
# shk_vel(params=theta, profile=2, plot=True)

In [ ]:
# profile 3

shk_vel(params=params[3], profile=3, plot=True)

In [ ]:
bounds2 = np.array([[0.04, 0.1], [0.4, 0.55], [0.15, 0.45], [0.8, 0.95], [0.05, 0.35]])

LH2 = LHC(sample_space=sample_space, bounds=bounds2,\
         calib_samp=500, GP_samp=500000)


In [ ]:
predict_samples = LH2.GP_samp_space

params = {}

for profile in profiles:
 
    log_L_p, V_p = GPs_log_L[profile].GP_predict(X = predict_samples)

    index = int(np.where(log_L_p == np.max(log_L_p))[0])
    print(index)

    params[profile] = predict_samples[index, :]


print(params)



In [ ]:
params

In [ ]:
shk_vel(params=params[1], profile=1, plot=True)


In [ ]:

shk_vel(params=params[2], profile=2, plot=True)



In [ ]:

shk_vel(params=params[3], profile=3, plot=True)

In [ ]:
GPs_ST = {}

ST_profiles = [2, 3]
ST_outputs = [ST_results_P2, ST_results_P3]

for profile, output in zip(ST_profiles, ST_outputs):
    
    GPs_ST[profile]  = GP_reg(kernel='Exponential', samples=LH_samples, \
                           output=output, train_frac=0.7)

In [ ]:
ST_fnames = ['GP_calib/ST_P2_GP.pickle', 'GP_calib/ST_P3_GP.pickle']
ST_titles = [r'Profile 2', r'Profile 3']

for profile, fname, title in zip(ST_profiles, ST_fnames, ST_titles):

    GPs_ST[profile].test_train()
    GPs_ST[profile].regression()
    GPs_ST[profile].m.pickle(fname)
    print(GPs_ST[profile].len_scales)
    GPs_ST[profile].test_train_plot(title = title, shock_time = True)


In [ ]:
predict_samples = LH.GP_samp_space

params_ST = {}

for profile in ST_profiles:

    ST_res_p, V_ST_p = GPs_ST[profile].GP_predict(X = predict_samples)

    target = 0.0

    ST_res_p = ST_res_p.reshape(len(ST_res_p))

    target_arr = np.abs(ST_res_p-target)

    index = target_arr.argmin()
    
    params_ST[profile] = predict_samples[index, :]


print(params_ST)

In [ ]:
# profile 2

shk_vel(params=params_ST[2], profile=2, plot=True, shock_time=True)

In [ ]:
# profile 3

shk_vel(params=params_ST[3], profile=3, plot=True, shock_time=True)

## Sensitivity Analysis

In [ ]:
x0 = []
sig = []

for i in range(5):
    x0.append(np.mean(predict_samples[:,i]))
    sig.append(np.std(predict_samples[:,i]))

## Log L results


In [ ]:
## GP sens
for profile in profiles:
    # if (profile == 1 or profile == 3):
    #     continue

    GPs_log_L[profile].GP_sensitivities(x0, plot = True, likelihood = True)